#### The purpose of this notebook is to demonstrate we need to be aware of skin colors when spliting training and validation dataset. This is a pretty big issue when it comes training. For example, a type pf skin disease occur on white and black skin color given by this dataset. If we only split the white skin color with this type of disease into training datset, we never get the black skin color correct in the validation dataset.

## Import Libraries

In [89]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import EfficientNetB2
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, concatenate, Flatten, Input, Concatenate
from keras import layers, models, regularizers, Model, optimizers
from keras.losses import CategoricalCrossentropy, CategoricalFocalCrossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Image Size
target_size = (260, 260)

# Batch Size
batch_size = 32

# Number of predictors
num_classes = 21

## Read data frame and Image Sampling

In [91]:
path = "../bttai-ajl-2025/train/train"
df = pd.read_csv("../bttai-ajl-2025/train.csv")

df['md5hash'] = df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
df['file_path'] = df['label'] + '/' + df['md5hash']

In [92]:
df.columns

Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'three_partition_label', 'qc', 'ddi_scale',
       'file_path'],
      dtype='object')

In [93]:
df.groupby(['three_partition_label', 'nine_partition_label', 'label']).size()

three_partition_label  nine_partition_label          label                             
benign                 benign-dermal                 dermatofibroma                         55
                                                     pyogenic-granuloma                     79
                       benign-epidermal              epidermal-nevus                        64
                                                     prurigo-nodularis                     119
                                                     seborrheic-keratosis                   48
malignant              malignant-cutaneous-lymphoma  mycosis-fungoides                     127
                       malignant-dermal              kaposi-sarcoma                        109
                       malignant-epidermal           actinic-keratosis                     122
                                                     basal-cell-carcinoma                  328
                                                     basa

In [ ]:
# Drop the image labelled wrongly
df = df[df["qc"] != "3 Wrongly labelled"].copy()


In [ ]:
# Map the values in the qc to integers

df.qc = df.qc.map({
    "1 Diagnostic": 4,
    "5 Potentially": 3,
    "2 Caracteristic": 2,
    "4 Other": 1
}).fillna(0).astype(int)


In [96]:
df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,0,34,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,0,12,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,4,56,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,0,34,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,0,12,prurigo-nodularis/c77d6c895f05fea73a8f37043070...


In [ ]:
# Since images with a number of 4, 3, 2, 1 are verified by experts already, we should put all these images into training

prioritized_df = df[df.qc.isin([4, 3, 2, 1])].copy()
prioritized_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,4,56,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
26,e99ba7397c33ba169192ffdb25b66ccf.jpg,5,2,seborrheic-keratosis,benign-epidermal,benign,4,56,seborrheic-keratosis/e99ba7397c33ba169192ffdb2...
75,6be528e219f8ce45c9782e2b05ae3c24.jpg,3,2,basal-cell-carcinoma,malignant-epidermal,malignant,4,34,basal-cell-carcinoma/6be528e219f8ce45c9782e2b0...
93,31749f92677e70999c28fe49fbc6dafc.jpg,2,2,eczema,inflammatory,non-neoplastic,4,12,eczema/31749f92677e70999c28fe49fbc6dafc.jpg
193,67c9271813f07f5311e2cb435a817403.jpg,2,3,basal-cell-carcinoma,malignant-epidermal,malignant,4,12,basal-cell-carcinoma/67c9271813f07f5311e2cb435...


In [ ]:
# Get the unverified images

unpriporitized_df = df[~df.qc.isin([4, 3, 2, 1])].copy()
unpriporitized_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,0,34,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,0,12,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,0,34,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,0,12,prurigo-nodularis/c77d6c895f05fea73a8f37043070...
5,9d5a90fa3f6934608add10e698001760.jpg,3,5,prurigo-nodularis,benign-epidermal,benign,0,34,prurigo-nodularis/9d5a90fa3f6934608add10e69800...


In [ ]:
# shuffle the data frame

prioritized_df = prioritized_df.sample(frac=1).reset_index(drop=True)
prioritized_df

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,bc578ce738482dd8f76af0c5ef32bca6.jpg,3,3,squamous-cell-carcinoma,malignant-epidermal,malignant,4,34,squamous-cell-carcinoma/bc578ce738482dd8f76af0...
1,454e68bf5705eabf0f4d28fc1dbd77be.jpg,6,1,superficial-spreading-melanoma-ssm,malignant-melanoma,malignant,4,56,superficial-spreading-melanoma-ssm/454e68bf570...
2,850f18a45b3fcbcd8b5020cc07273d91.jpg,3,4,basal-cell-carcinoma,malignant-epidermal,malignant,4,34,basal-cell-carcinoma/850f18a45b3fcbcd8b5020cc0...
3,e642ea55ce353ec974309c87f4da7373.jpg,5,-1,kaposi-sarcoma,malignant-dermal,malignant,4,56,kaposi-sarcoma/e642ea55ce353ec974309c87f4da737...
4,96e320998184af49bfc791c8f1b0e174.jpg,3,4,melanoma,malignant-melanoma,malignant,4,34,melanoma/96e320998184af49bfc791c8f1b0e174.jpg
...,...,...,...,...,...,...,...,...,...
79,f03c8430ae6b97cb1ee43fa8c78a89af.jpg,6,6,dyshidrotic-eczema,inflammatory,non-neoplastic,4,56,dyshidrotic-eczema/f03c8430ae6b97cb1ee43fa8c78...
80,67c9271813f07f5311e2cb435a817403.jpg,2,3,basal-cell-carcinoma,malignant-epidermal,malignant,4,12,basal-cell-carcinoma/67c9271813f07f5311e2cb435...
81,bc00bb114e7236974ea26ae39b97e563.jpg,3,3,dermatomyositis,inflammatory,non-neoplastic,4,34,dermatomyositis/bc00bb114e7236974ea26ae39b97e5...
82,203f72e12bfd9132ba8f843eaa9a894f.jpg,2,1,melanoma,malignant-melanoma,malignant,4,12,melanoma/203f72e12bfd9132ba8f843eaa9a894f.jpg


In [ ]:
# Create a new col named skinColor_combined_label
# this new column is combined with different skin colors asscoiated with different type of skin disease
# As we train test split datatset by stratify skinColor_combined_label, this ensures we don't have only white skin in the training, and black skin in validation

unpriporitized_df["skinColor_combined_label"] = unpriporitized_df["fitzpatrick_scale"].astype(str) + "_" + unpriporitized_df["three_partition_label"] + "_" + unpriporitized_df["nine_partition_label"] + "_" + unpriporitized_df["label"]

In [ ]:
# Get rarer skin colors asscoiated with a skin disease
indices = unpriporitized_df.skinColor_combined_label.value_counts() == 1
rare_disease = list(unpriporitized_df.skinColor_combined_label.value_counts().index[indices])

In [102]:
rare_disease

['-1_benign_benign-epidermal_seborrheic-keratosis',
 '6_malignant_malignant-melanoma_superficial-spreading-melanoma-ssm',
 '5_malignant_malignant-epidermal_basal-cell-carcinoma-morpheiform',
 '-1_benign_benign-epidermal_prurigo-nodularis',
 '6_malignant_malignant-melanoma_malignant-melanoma',
 '-1_malignant_malignant-epidermal_basal-cell-carcinoma-morpheiform',
 '-1_benign_benign-dermal_dermatofibroma']

In [103]:
small_df = unpriporitized_df[unpriporitized_df.skinColor_combined_label.isin(rare_disease)].copy()
unpriporitized_df = unpriporitized_df[~unpriporitized_df.skinColor_combined_label.isin(rare_disease)].copy()

In [104]:
small_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path,skinColor_combined_label
27,71b67be202663f843f95b5f409b358e4.jpg,-1,-1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,0,-1,basal-cell-carcinoma-morpheiform/71b67be202663...,-1_malignant_malignant-epidermal_basal-cell-ca...
80,e815861f140e305baf441814e6dbda48.jpg,6,1,malignant-melanoma,malignant-melanoma,malignant,0,56,malignant-melanoma/e815861f140e305baf441814e6d...,6_malignant_malignant-melanoma_malignant-melanoma
472,31a8db3e9da7907512beaa575d638be4.jpg,5,3,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,0,56,basal-cell-carcinoma-morpheiform/31a8db3e9da79...,5_malignant_malignant-epidermal_basal-cell-car...
748,9e8595b4c1edec4b70653523997c267f.jpg,-1,1,seborrheic-keratosis,benign-epidermal,benign,0,-1,seborrheic-keratosis/9e8595b4c1edec4b706535239...,-1_benign_benign-epidermal_seborrheic-keratosis
1177,02f11821915d67f20fb15a8e8b96c1d3.jpg,6,1,superficial-spreading-melanoma-ssm,malignant-melanoma,malignant,0,56,superficial-spreading-melanoma-ssm/02f11821915...,6_malignant_malignant-melanoma_superficial-spr...


In [ ]:
# Train Test Split

unpriporitized_train_df, unpriporitized_validation_df = train_test_split(unpriporitized_df, 
                                                                         test_size = 0.2, 
                                                                         stratify=unpriporitized_df.skinColor_combined_label,
                                                                         random_state=42)

In [ ]:
# Concatenate training data

train_df = pd.concat([prioritized_df, small_df, unpriporitized_train_df]).copy().reset_index(drop=True)
train_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path,skinColor_combined_label
0,bc578ce738482dd8f76af0c5ef32bca6.jpg,3,3,squamous-cell-carcinoma,malignant-epidermal,malignant,4,34,squamous-cell-carcinoma/bc578ce738482dd8f76af0...,NaN
1,454e68bf5705eabf0f4d28fc1dbd77be.jpg,6,1,superficial-spreading-melanoma-ssm,malignant-melanoma,malignant,4,56,superficial-spreading-melanoma-ssm/454e68bf570...,NaN
2,850f18a45b3fcbcd8b5020cc07273d91.jpg,3,4,basal-cell-carcinoma,malignant-epidermal,malignant,4,34,basal-cell-carcinoma/850f18a45b3fcbcd8b5020cc0...,NaN
3,e642ea55ce353ec974309c87f4da7373.jpg,5,-1,kaposi-sarcoma,malignant-dermal,malignant,4,56,kaposi-sarcoma/e642ea55ce353ec974309c87f4da737...,NaN
4,96e320998184af49bfc791c8f1b0e174.jpg,3,4,melanoma,malignant-melanoma,malignant,4,34,melanoma/96e320998184af49bfc791c8f1b0e174.jpg,NaN


In [112]:
train_df.label.value_counts()

label
squamous-cell-carcinoma               326
basal-cell-carcinoma                  265
folliculitis                          191
acne-vulgaris                         188
melanoma                              146
eczema                                115
acne                                  103
mycosis-fungoides                     103
actinic-keratosis                      97
prurigo-nodularis                      96
kaposi-sarcoma                         89
keloid                                 88
dermatomyositis                        86
superficial-spreading-melanoma-ssm     69
pyogenic-granuloma                     64
malignant-melanoma                     62
epidermal-nevus                        52
dyshidrotic-eczema                     46
dermatofibroma                         44
seborrheic-keratosis                   38
basal-cell-carcinoma-morpheiform       35
Name: count, dtype: int64

In [113]:
unpriporitized_validation_df.label.value_counts()

label
squamous-cell-carcinoma               81
basal-cell-carcinoma                  63
acne-vulgaris                         46
folliculitis                          46
melanoma                              35
eczema                                28
actinic-keratosis                     25
acne                                  24
mycosis-fungoides                     24
prurigo-nodularis                     23
keloid                                21
kaposi-sarcoma                        20
dermatomyositis                       20
malignant-melanoma                    16
pyogenic-granuloma                    15
superficial-spreading-melanoma-ssm    14
dyshidrotic-eczema                    12
epidermal-nevus                       12
dermatofibroma                        11
seborrheic-keratosis                   9
basal-cell-carcinoma-morpheiform       8
Name: count, dtype: int64

## ImageDataGenerator

In [ ]:
# Create image data geneerator to read images and apply data agumentation to only training dataset

train_datagen = ImageDataGenerator(
    # we have to use efficient net preprocessing lib to be consistent
    preprocessing_function=keras.applications.efficientnet.preprocess_input,
    rotation_range = 30,
    shear_range=0.2,
    zoom_range=0.2,
    # we may not need this if we use efficient net since we have to resize anyway
    # rescale = 1./255,
    horizontal_flip = True,
    vertical_flip = True,
    brightness_range=[0.7, 1.2],
    height_shift_range = 0.1,
    width_shift_range = 0.1
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=keras.applications.efficientnet.preprocess_input
)

# flow the data into the datagens
train_data = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = path,
    x_col = "file_path",
    y_col = "label",
    target_size = target_size,
    batch_size = batch_size,
    class_mode = "categorical",
    seed = 42
)

val_data = validation_datagen.flow_from_dataframe(
    dataframe = unpriporitized_validation_df,
    directory = path,
    x_col = "file_path",
    y_col = "label",
    target_size = target_size,
    batch_size = batch_size,
    class_mode = "categorical",
    seed=42
)

Found 2303 validated image filenames belonging to 21 classes.
Found 553 validated image filenames belonging to 21 classes.


In [117]:
train_data.class_indices

{'acne': 0,
 'acne-vulgaris': 1,
 'actinic-keratosis': 2,
 'basal-cell-carcinoma': 3,
 'basal-cell-carcinoma-morpheiform': 4,
 'dermatofibroma': 5,
 'dermatomyositis': 6,
 'dyshidrotic-eczema': 7,
 'eczema': 8,
 'epidermal-nevus': 9,
 'folliculitis': 10,
 'kaposi-sarcoma': 11,
 'keloid': 12,
 'malignant-melanoma': 13,
 'melanoma': 14,
 'mycosis-fungoides': 15,
 'prurigo-nodularis': 16,
 'pyogenic-granuloma': 17,
 'seborrheic-keratosis': 18,
 'squamous-cell-carcinoma': 19,
 'superficial-spreading-melanoma-ssm': 20}

In [118]:
val_data.class_indices

{'acne': 0,
 'acne-vulgaris': 1,
 'actinic-keratosis': 2,
 'basal-cell-carcinoma': 3,
 'basal-cell-carcinoma-morpheiform': 4,
 'dermatofibroma': 5,
 'dermatomyositis': 6,
 'dyshidrotic-eczema': 7,
 'eczema': 8,
 'epidermal-nevus': 9,
 'folliculitis': 10,
 'kaposi-sarcoma': 11,
 'keloid': 12,
 'malignant-melanoma': 13,
 'melanoma': 14,
 'mycosis-fungoides': 15,
 'prurigo-nodularis': 16,
 'pyogenic-granuloma': 17,
 'seborrheic-keratosis': 18,
 'squamous-cell-carcinoma': 19,
 'superficial-spreading-melanoma-ssm': 20}

## Compute Class Weights

In [ ]:
# Compute weights to mitigate imbalance issues

train_classes = train_data.classes
cw = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_classes),
    y=train_classes
)
class_weights = dict(enumerate(cw))

print("Class Weights:", class_weights)

Class Weights: {0: 1.064724919093851, 1: 0.5833333333333334, 2: 1.1305841924398625, 3: 0.4138364779874214, 4: 3.1333333333333333, 5: 2.492424242424242, 6: 1.2751937984496124, 7: 2.3840579710144927, 8: 0.9536231884057971, 9: 2.108974358974359, 10: 0.5741710296684118, 11: 1.2322097378277153, 12: 1.246212121212121, 13: 1.7688172043010753, 14: 0.7511415525114156, 15: 1.064724919093851, 16: 1.1423611111111112, 17: 1.7135416666666667, 18: 2.8859649122807016, 19: 0.33640081799591004, 20: 1.5893719806763285}


## Modeling

In [ ]:
# Use efficientnetB2 

base_model = EfficientNetB2(
    include_top=False,
    weights="imagenet",
    input_shape=(260, 260, 3)
)

# directly fine tune the model for our case
base_model.trainable = True

In [ ]:
# Model Structure

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)

# First Dense Layer with BatchNorm and L2 Regularization
# x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
# x = layers.BatchNormalization()(x)  # Helps stabilize training
# x = layers.Dropout(0.3)(x)  # Reduce to avoid excessive information loss

# Second Dense Layer for more representation
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)  

outputs = layers.Dense(num_classes, activation="softmax")(x)

# make/define the model
model = models.Model(inputs=base_model.input, outputs=outputs)

In [ ]:
# Use catgorical Focal Cross entropy to handle multiclass problems
loss = CategoricalFocalCrossentropy(label_smoothing=0.1)

optimizer = optimizers.Adam(learning_rate=0.0003)

model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=['accuracy']
             )

In [128]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
]

In [129]:
history = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=callbacks,
    class_weight=class_weights
)

Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.0887 - loss: 1.3700

c:\Users\42128\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


72/72 ━━━━━━━━━━━━━━━━━━━━ 359s 4s/step - accuracy: 0.0892 - loss: 1.3689 - val_accuracy: 0.2315 - val_loss: 1.0180 - learning_rate: 3.0000e-04
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 296s 4s/step - accuracy: 0.2552 - loss: 1.0600 - val_accuracy: 0.3562 - val_loss: 0.9328 - learning_rate: 3.0000e-04
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 294s 4s/step - accuracy: 0.3542 - loss: 0.9604 - val_accuracy: 0.3725 - val_loss: 0.8838 - learning_rate: 3.0000e-04
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 293s 4s/step - accuracy: 0.4001 - loss: 0.8648 - val_accuracy: 0.4448 - val_loss: 0.8559 - learning_rate: 3.0000e-04
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 292s 4s/step - accuracy: 0.4498 - loss: 0.8017 - val_accuracy: 0.4792 - val_loss: 0.8521 - learning_rate: 3.0000e-04
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 295s 4s/step - accuracy: 0.5117 - loss: 0.7491 - val_accuracy: 0.4792 - val_loss: 0.8276 - learning_rate: 3.0000e-04
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 286s 4s/step - accuracy: 0.5957 - loss: 0.7

In [ ]:
# model.save("All_Label_Model1.keras")

## Testing

In [201]:
test_df = pd.read_csv("../bttai-ajl-2025/test.csv")

test_path = "../bttai-ajl-2025/test/test/"

# Combine label and md5hash to form the correct path
test_df['file_path'] = test_path + test_df['md5hash'] + '.jpg'

In [202]:
test_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,qc,ddi_scale,file_path
0,0844ae634f0e6e7ef1f73c2aeecbae0e,2,2,NaN,12,../bttai-ajl-2025/test/test/0844ae634f0e6e7ef1...
1,3b290d262098f761d719aa07cf36c040,4,3,NaN,34,../bttai-ajl-2025/test/test/3b290d262098f761d7...
2,cf561d08ac46d0fda678bff6621005ee,2,3,NaN,12,../bttai-ajl-2025/test/test/cf561d08ac46d0fda6...
3,e6371069be05c6b0a95b4b3f1bacc9a5,4,3,NaN,34,../bttai-ajl-2025/test/test/e6371069be05c6b0a9...
4,f76cddb37265f97508f159078dcc7e7c,5,5,NaN,56,../bttai-ajl-2025/test/test/f76cddb37265f97508...


In [ ]:
# For submission, you have to follow image order in test.csv

test_datagen = ImageDataGenerator(preprocessing_function=keras.applications.efficientnet.preprocess_input)

# Load test images (Ensure they are all inside a single folder, not subdirectories)
test_data = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="file_path",      # Column containing file paths
    target_size=target_size, # Resize images (adjust for your model)
    batch_size=batch_size,          # Batch size
    class_mode=None,        # No labels for test set
    shuffle=False           # Keep order consistent
)

Found 1227 validated image filenames.


In [193]:
predictions = model.predict(test_data)

c:\Users\42128\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 26s 619ms/step


In [194]:
predicted_classes = np.argmax(predictions, axis=1)

In [195]:
index_to_class = {v: k for k, v in train_data.class_indices.items()}

In [196]:
index_to_class

{0: 'acne',
 1: 'acne-vulgaris',
 2: 'actinic-keratosis',
 3: 'basal-cell-carcinoma',
 4: 'basal-cell-carcinoma-morpheiform',
 5: 'dermatofibroma',
 6: 'dermatomyositis',
 7: 'dyshidrotic-eczema',
 8: 'eczema',
 9: 'epidermal-nevus',
 10: 'folliculitis',
 11: 'kaposi-sarcoma',
 12: 'keloid',
 13: 'malignant-melanoma',
 14: 'melanoma',
 15: 'mycosis-fungoides',
 16: 'prurigo-nodularis',
 17: 'pyogenic-granuloma',
 18: 'seborrheic-keratosis',
 19: 'squamous-cell-carcinoma',
 20: 'superficial-spreading-melanoma-ssm'}

In [197]:
predicted_class_names = [index_to_class[idx] for idx in predicted_classes]

In [200]:
test_df.columns

Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'qc',
       'ddi_scale', 'file_path'],
      dtype='object')

In [203]:
test_submission = test_df.drop(columns=['fitzpatrick_scale', 'fitzpatrick_centaur', 'qc',
       'ddi_scale', 'file_path']).copy()

test_submission["label"] = predicted_class_names

In [204]:
test_submission.head()

,md5hash,label
0,0844ae634f0e6e7ef1f73c2aeecbae0e,prurigo-nodularis
1,3b290d262098f761d719aa07cf36c040,actinic-keratosis
2,cf561d08ac46d0fda678bff6621005ee,squamous-cell-carcinoma
3,e6371069be05c6b0a95b4b3f1bacc9a5,acne-vulgaris
4,f76cddb37265f97508f159078dcc7e7c,folliculitis


In [205]:
test_submission.to_csv("../Submission/test.csv", index=False)

In [206]:
test_submission.head()

,md5hash,label
0,0844ae634f0e6e7ef1f73c2aeecbae0e,prurigo-nodularis
1,3b290d262098f761d719aa07cf36c040,actinic-keratosis
2,cf561d08ac46d0fda678bff6621005ee,squamous-cell-carcinoma
3,e6371069be05c6b0a95b4b3f1bacc9a5,acne-vulgaris
4,f76cddb37265f97508f159078dcc7e7c,folliculitis
